In [1]:
import os
import time
import sys

import anuga

from math import sin, pi, exp

In [2]:
# Create DEM from asc data
anuga.asc2dem('11f3.asc', use_cache=False, verbose=True)

# Create DEM from asc data
anuga.dem2pts('11f3.dem', use_cache=False, 
              verbose=True)

# Create roughness raster from asc data
#anuga.asc2dem('11_man_c.asc', use_cache=False, verbose=True)

# Create roughness raster from asc data
#anuga.dem2pts('11_man_c.dem', use_cache=False, 
#              verbose=True)

Reading METADATA from 11f3.prj
Reading DEM from 11f3.asc
Got 542 lines
Store to NetCDF file 11f3.dem
Reading DEM from 11f3.dem
Store to NetCDF file 11f3.pts
There are 581560 values in the elevation
There are 581560 values in the clipped elevation
There are 0 NODATA_values in the clipped elevation


In [3]:
bounding_polygon = anuga.read_polygon('extent.csv')
domain = anuga.create_domain_from_regions(bounding_polygon,
                                    boundary_tags={'top': [0],
                                                   'right': [1],
                                                   'bottom': [2],
                                                   'left': [3]},
                                    maximum_triangle_area=1,
                                    mesh_filename='11f.msh',
                                    use_cache=False,
                                    verbose=True)

#domain = anuga.Domain(points, vertices, boundary)
domain.set_name('11f3_NoMan')
domain.set_datadir('.')

domain.set_quantity('elevation', filename='11f3.pts')
#domain.set_quantity('friction',filename='11_man_c.pts')
domain.set_quantity('friction',0.261)
domain.set_quantity('stage',expression='elevation')  # DRY BED

Quantity(domain, name = 'veg_diameter', register = True)
domain.set_quantity('veg_diameter', 0.50)

Quantity(domain, name = 'veg_sspacing', register = True)
domain.set_quantity('veg_diameter', 0.00064)



Generating mesh to file '11f.msh'
Domain: Initialising
Pmesh_to_Domain: Initialising
Pmesh_to_Domain: Done
General_mesh: Building basic mesh structure
General_mesh: Computing areas, normals, edgelengths, centroids and radii
General Mesh: Building inverted triangle structure
Mesh: Initialising
Mesh: Building neigbour structure
Mesh: Building surrogate neigbour structure
Mesh: Building boundary dictionary
Mesh: Building tagged elements dictionary
Mesh: Done
Domain: Expose mesh attributes
Domain: Expose quantity names and types
Domain: Build Quantities
Domain: Set up communication buffers 
Domain: Set up triangle/node full flags 
Domain: Set defaults
Domain: Set work arrays
Domain: Initialising quantity values
Domain: Done
##########################################################################
#
# Using discontinuous elevation solver DE0
#
# First order timestepping
#
# Make sure you use centroid values when reporting on important output quantities
#
###################################

In [4]:
Br = anuga.Reflective_boundary(domain)      # Solid reflective wall
Bt = anuga.Transmissive_boundary(domain)    # Continue all values on boundary


domain.set_boundary({'left': Br, 'right': Bt, 'top': Br, 'bottom': Br})

# Setup inlet flow
center = (538416.0, 4190718.0)
radius = 10.0
region0 = anuga.Region(domain, center=center, radius=radius)
fixed_inflow = anuga.Inlet_operator(domain, region0 , Q=20)

In [ ]:
for t in domain.evolve(yieldstep=500, finaltime=6500):
    print domain.timestepping_statistics()

Time = 0.0000, steps=0 (146s)
Time = 500.0000, delta t = 1000.00000000, steps=1 (1s)
Time = 1000.0000, delta t in [0.02051730, 0.10539335], steps=5148 (4427s)
Time = 1500.0000, delta t in [0.09306568, 0.09739862], steps=5270 (4604s)
Time = 2000.0000, delta t in [0.09739906, 0.09850080], steps=5092 (4191s)
Time = 2500.0000, delta t in [0.07164000, 0.09852990], steps=6062 (5000s)
